# NNLM (Neural Network Language Model)

In [181]:
import torch
import torch.nn as nn
import torch.optim as optim

In [182]:
class NNLM(nn.Module):                                      # 신경망 언어 모델 클래스 정의
    def __init__(self, vocab_size, embed_size, hidden_size, context_size):
        super(NNLM, self).__init__()                         # nn.Module 초기화
        self.embed = nn.Embedding(vocab_size, embed_size)   # 단어 ID → 임베딩 벡터 변환
        self.fc1 = nn.Linear(context_size * embed_size, hidden_size)
                                                             # context 단어 임베딩을 펼쳐서 hidden layer로
        self.relu = nn.ReLU()                                # 비선형 활성화 함수
        self.fc2 = nn.Linear(hidden_size, vocab_size)        # hidden → 전체 단어 분포
        self.log_softmax = nn.LogSoftmax(dim=1)              # 로그 확률 형태로 정규화

    def forward(self, x):
        embeds = self.embed(x)                               # 입력 단어 ID → 임베딩 (batch, context, embed)
        embeds = embeds.view(embeds.size(0), -1)             # 임베딩을 1차원으로 펼침
        output = self.fc1(embeds)                            # 첫 번째 선형 변환
        output = self.relu(output)                           # ReLU 적용
        output = self.fc2(output)                            # 단어별 점수(logit) 계산
        log_probs = self.log_softmax(output)                 # 다음 단어의 로그 확률 분포
        return log_probs                                     # 최종 출력 반환

In [183]:
VOCAB_SIZE = 5000                        # 전체 단어 사전 크기 (단어 ID 개수)
EMBED_SIZE = 300                        # 각 단어를 표현할 임베딩 벡터 차원
HIDDEN_SIZE = 128                       # 은닉층(hidden layer) 뉴런 수
CONTEXT_SIZE = 2                        # 입력으로 사용할 이전 단어 개수

model = NNLM(                           # NNLM 모델 생성
    VOCAB_SIZE,                         # 단어 사전 크기 전달
    EMBED_SIZE,                         # 임베딩 차원 전달
    HIDDEN_SIZE,                        # 은닉층 크기 전달
    CONTEXT_SIZE                        # 문맥 단어 수 전달
)

print(model)                            # 모델 구조(레이어 구성) 출력


NNLM(
  (embed): Embedding(5000, 300)
  (fc1): Linear(in_features=600, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=5000, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)


In [184]:
# dummy data 생성
X = torch.randint(0, VOCAB_SIZE, (8, CONTEXT_SIZE))   # 입력 데이터: 배치 8개, 각 샘플마다 context 단어 ID 2개
y = torch.randint(0, VOCAB_SIZE, (8,))                # 정답 데이터: 각 입력에 대한 다음 단어 ID (총 8개_


In [185]:
criterion = nn.NLLLoss()                     # 로그 확률 출력에 사용하는 손실 함수 (Negative Log Likelihood)
optimizer = optim.Adam(
    model.parameters(),                     # 모델의 모든 파라미터를 최적화 대상으로 설정
    lr=0.001                                # 학습률
)

model.train()                               # 모델을 학습 모드로 전환
optimizer.zero_grad()                       # 이전 step의 gradient 초기화
output = model(X)                           # 입력 X로 예측된 로그 확률 계산
loss = criterion(output, y)                 # 예측과 정답(y)으로 손실 계산
loss.backward()                             # 역전파로 gradient 계산
optimizer.step()                            # 파라미터 업데이트

print(loss.item())                          # 현재 step의 loss 값 출력


8.476271629333496
